# Исследование надёжности заёмщиков
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem 
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
m = Mystem()
df = pd.read_csv('~/GitHub/lesson2/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
display(df.sample(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
11096,0,-2315.370850,42,среднее,1,гражданский брак,1,M,компаньон,0,139192.845977,сделка с подержанным автомобилем
1672,1,-1190.566743,54,среднее,1,гражданский брак,1,F,сотрудник,0,106436.116561,автомобили
5315,20,-2047.754733,24,среднее,1,женат / замужем,0,F,сотрудник,0,100415.236833,покупка коммерческой недвижимости
13085,0,330106.771115,62,среднее,1,женат / замужем,0,F,пенсионер,0,114119.849198,на покупку автомобиля
15109,0,-244.182441,21,среднее,1,Не женат / не замужем,4,M,сотрудник,0,164669.944295,операции со своей недвижимостью
17262,0,-207.676669,39,среднее,1,вдовец / вдова,2,F,компаньон,0,135163.190100,дополнительное образование
2954,0,-1417.570449,29,высшее,0,гражданский брак,1,F,компаньон,0,392135.657594,сыграть свадьбу
15581,0,-185.263416,34,высшее,0,женат / замужем,0,F,компаньон,0,267472.357738,на покупку автомобиля
1054,1,NaN,59,среднее,1,женат / замужем,0,M,компаньон,1,NaN,высшее образование
9017,1,-1457.500801,44,СРЕДНЕЕ,1,гражданский брак,1,F,сотрудник,1,157709.904672,на покупку автомобиля


**Вывод**

Дата-фрейм имеет 12 колонок и 21 525 строк. В столбцах days_employed и total_income присустствуют пропуски, так как в них содержится 19 351 строках. Скорее всего они находяться в одних и тех же строчках. И соответственно возможно что в датасете присутствуют не платежеспособные клиенты (так как нет стажа и дохода), следовательно не целесообразно им давать кредит. Но наверняка это не известно. Но стоит обратить на это внимание.

# 2. Предобработка данных

### Обработка пропущенных значений ежемесячного дохода

- Заменим пустые значения в ежемесячном доходе на медианные по каждому типу занятости
- Для этого проверим, в каких типах занятости встречаются пропущенные значения доходов

In [3]:
df.loc[df['total_income'].isnull(), 'income_type'].value_counts()

income_type
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: count, dtype: int64

- Найдем медианные уровни дохода для каждого типа занятости

In [4]:
medians_total_income = df.groupby('income_type')['total_income'].median().round()
medians_total_income

income_type
безработный        131340.0
в декрете           53829.0
госслужащий        150448.0
компаньон          172358.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98202.0
Name: total_income, dtype: float64

Медианный доход различается по группам - это может говорить о том, что выбранная характеристика (тип занятости) вполне описывает уровень дохода. Таким способом можно группировать по разным характеристикам или их сочетанию, чтобы найти наилучший вариант.

- Заменим пропущенные значения ежемесячного дохода на медианы по типам занятости.

In [5]:
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = medians_total_income[6]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = medians_total_income[3]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = medians_total_income[4]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = medians_total_income[2]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = medians_total_income[5]

### Исправление нулевых значений в возрасте клиентов

- Посмотрим, какой тип занятости имеют клиенты с ошибочным возрастом

In [6]:
df.loc[df['dob_years'] == 0, 'income_type'].value_counts()

income_type
сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: count, dtype: int64

- Всего их 101 человек, большая часть из которых являются сотрудниками организаций
- Найдем медианные значения возрастов клиентов для каждого типа занятости

In [7]:
age_medians = df.groupby('income_type')['dob_years'].median()
age_medians

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

- Теперь заменим нулевые значения на медианы по каждому типу занятости

In [8]:
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'сотрудник'), 'dob_years'] = age_medians[6]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'пенсионер'), 'dob_years'] = age_medians[4]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'компаньон'), 'dob_years'] = age_medians[3]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'госслужащий'), 'dob_years'] = age_medians[2]

- Проверим, все ли нули мы исправили

In [9]:
df.loc[df['dob_years'] == 0, 'dob_years'].value_counts()

Series([], Name: count, dtype: int64)

Обработка пропущенных значений трудового стажа

- Проверим количество пропущенных данных в столбце

In [10]:
df.query('days_employed != days_employed & total_income != total_income').isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [11]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


- Избавимся от отрицательных значений в трудовом стаже

In [12]:
df['days_employed'] = df['days_employed'].abs()

In [13]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


- Найдем медианное значение трудового стажа для каждой возрастной группы

In [14]:
age_statistics = df['dob_years'].describe()
age_statistics[4:7]

25%    34.0
50%    43.0
75%    53.0
Name: dob_years, dtype: float64

- Напишем функцию, которая принимает на вход возраст клиента и возвращает возрастную категорию

In [15]:
def age_group(age):
    if age <= age_statistics[4]: return 'до 34'
    elif age_statistics[4] < age <= age_statistics[5]: return '34-43'
    elif age_statistics[5] < age <= age_statistics[6]: return '43-53'
    else: return '53+'
    
df['age_group'] = df['dob_years'].apply(age_group)
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
21064,0,400655.639661,55,НАЧАЛЬНОЕ,3,женат / замужем,0,F,пенсионер,0,58574.645855,на покупку подержанного автомобиля,53+
1385,2,1583.917264,42,высшее,0,Не женат / не замужем,4,F,сотрудник,0,131237.854036,операции с жильем,34-43
17425,0,396783.883705,65,среднее,1,женат / замужем,0,F,пенсионер,0,120571.640970,сделка с подержанным автомобилем,53+
4184,0,376617.584253,57,среднее,1,женат / замужем,0,F,пенсионер,0,165644.100462,дополнительное образование,53+
17017,3,1550.274427,35,среднее,1,женат / замужем,0,F,сотрудник,0,94831.758709,недвижимость,34-43


- Найдем медианное значение трудового стажа для каждой возрастной группы

In [16]:
medians_of_days_employed = df.groupby('age_group')['days_employed'].median()
round(medians_of_days_employed, 0)

age_group
34-43      1838.0
43-53      2510.0
53+      343034.0
до 34      1179.0
Name: days_employed, dtype: float64

- Заменим пропущенные значения трудового стажа на медиану из возрастных категорий

In [17]:
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '34-43'), 'days_employed'] = medians_of_days_employed[0]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '43-53'), 'days_employed'] = medians_of_days_employed[1]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == '53+'), 'days_employed'] = medians_of_days_employed[2]
df.loc[(df['days_employed'].isnull()) & (df['age_group'] == 'до 34'), 'days_employed'] = medians_of_days_employed[3]

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  age_group         21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


- Проверим на уникальные значения столбец 'children'

In [19]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

- В столбце 'children' есть значения -1 и 20, явно ошибочные
- Заменим ошибочные значения в количестве детей из следующего предположения:
- вместо 20 детей укажем 2-х, вместо -1 укажем 1.

In [20]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1

- Проверим значения в столбце пол клиента. В целом данный столбец не влияет на аналитику, и наши выводы. Но для красоты исправим ошибки и тут

In [21]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

- Строка с пропущенным значением всего одна. Ее можно исключить из выборки. Так как нет дополнительных данных по которым мы бы смогли идентифицировать пол клиента (например по ФИО).

In [22]:
df = df[df['gender'] != 'XNA']

**Вывод**

Все строки заполнены, обработаны пропущенные значения по ежемесячному доходу на основе медиан для каждого типа занятости. Обработаны пропущенные значения в данных по трудовому стажу на основе медиан для каждой возрастной категории. Исправлены значения нулевого возраста на медианы соответствующего типа занятости Исправлены значения 

### Замена типа данных

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21524 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     21524 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  float64
 11  purpose           21524 non-null  object 
 12  age_group         21524 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB


В столбцах присутствует тип float. Это дробный тип чисел. При работе с такими данными как зарплата и трудовой стаж будет лучше и удобнее если они будут представлены в виде целого числа, в данном случае в формате int. 

In [24]:
df['total_income'] = df['total_income'].astype(int)
df['days_employed'] = df['days_employed'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21524 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   days_employed     21524 non-null  int64 
 2   dob_years         21524 non-null  int64 
 3   education         21524 non-null  object
 4   education_id      21524 non-null  int64 
 5   family_status     21524 non-null  object
 6   family_status_id  21524 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21524 non-null  object
 9   debt              21524 non-null  int64 
 10  total_income      21524 non-null  int64 
 11  purpose           21524 non-null  object
 12  age_group         21524 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.3+ MB


### Обработка дубликатов

- Перед поиском дубликатов приведем все данные по образованию клиентов к нижнему регистру. В противном случае, при поиске дубликатов значения "Среднее" и "среднее" будут определены, как 2 разных значения, что неверно.
- Посчитаем всех клиентов по уровням образования.

In [25]:
df['education'].value_counts()

education
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: count, dtype: int64

- Cделаем все буквы в значениях образования строчными

In [26]:
df['education'] = df['education'].str.lower()

- Посмотрим, что получилось, одновременно посчитая количество клиентов по каждому уровню образования

In [27]:
df['education'].value_counts()

education
среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: count, dtype: int64

- Посчитаем количество дубликатов в массиве

In [28]:
print('Число дубликатов в массиве данных:', df.duplicated().sum())
print('Число полностью идентичных строк:', df.duplicated(keep = False).sum())
print('Доля дубликатов из общей длины массива: {:.2%}'.format(df.duplicated().sum() / len(df)))

Число дубликатов в массиве данных: 71
Число полностью идентичных строк: 137
Доля дубликатов из общей длины массива: 0.33%


- Для наглядности выведем первые 5 строк полностью "одинаковых" клиентов

In [29]:
df[df.duplicated(keep = False)].sort_values('dob_years', ascending = False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
7938,0,343033,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
6537,0,343033,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
9604,0,343033,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,53+
5865,0,343033,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью,53+
9528,0,343033,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью,53+


- Удалим одинаковые строки

In [30]:
df = df.drop_duplicates().reset_index(drop = True)

- Проверим число дубликатов

In [31]:
print('Число дубликатов в массиве данных:', df.duplicated().sum())

Число дубликатов в массиве данных: 0


**Вывод**

- При обработке данных были найдены дубликаты (71 штука), это 0,33% от всего числа клиентов в датасете
- Необходимо выяснить, связано ли появление дубликатов с технической ошибкой, или же ошибка носит человеческий фактор
- При удалении дубликатов использовался метод drop_duplicates(), так как его целью является поиск идентичных строк
- Необходимо обратить внимание на метод заполнения ячеек с уровнем образования клиентов
- На данный момент ячейки заполняются вручную, что приводит к значениям вроде "среднее", "Среднее" или "СРЕДНЕЕ"
- На примере среднего образования: всего были неправильно заполнены данные по 1483 клиентам (9,74% от общего числа клиентов со средним образованием)
- Необходимо ввести условие на нижний регистр и выпадающий список всех возможных вариантов уровня образования

### Лемматизация

- Посчитаем все варианты целей кредита

In [32]:
df['purpose'].value_counts()

purpose
свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образовани

- Среди всех возможных целей получения кредита можно выделить всего 5 ключевых слов, которые отражают главную цель.
- Основные ключевые слова можно выделить вручную.

In [33]:
categories = ["сдача", "коммерческий", "жилье", "образование", "свадьба", "недвижимость", "автомобиль"]

- Проведем лемматизацию одновременно заменив полученный список лемм в каждой строке на главное ключевое слово из списка категорий.

In [34]:
def lemmatize(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma

df['purpose_group'] = df['purpose'].apply(lemmatize)        
df.head(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,автомобиль


- Посчитаем все возможные категории целей кредита.

In [35]:
df['purpose_group'].value_counts()

purpose_group
недвижимость    5039
автомобиль      4306
образование     4013
жилье           3809
свадьба         2324
коммерческий    1311
сдача            651
Name: count, dtype: int64

- Жилье тоже относится к недвижимости, поэтому сделаем соответствующую замену.
- Еще заменим категории по покупке коммерческой недвижимости или покупке жилой недвижимости для сдачи в наем на инвестиционную цель и снова посчитаем число категорий.

In [36]:
df.loc[df['purpose_group'] == 'жилье', 'purpose_group'] = 'недвижимость'
df.loc[(df['purpose_group'] == 'коммерческий') | (df['purpose_group'] == 'сдача'), 'purpose_group'] = 'инвестиционная цель'
df['purpose_group'].value_counts()

purpose_group
недвижимость           8848
автомобиль             4306
образование            4013
свадьба                2324
инвестиционная цель    1962
Name: count, dtype: int64

**Вывод**

Из цели получения кредита каждого клиента было выделено по одному ключевому слову:

- цель на покупку своего автомобиля заменяется на автомобиль.
- цель покупка жилой недвижимости заменяется на недвижимость.
- и т.д..

Подобное выделение ключевых слов из целей клиентов упрощает дальнейший анализ и сохраняет основной смысл цели получения кредита.

Все цели, содержащие производные слова от слова жилье (3809 чел.), были заменены на категорию недвижимость (цели с исходным ключевым словом насчитывают 5040 чел.), кроме целей по покупке коммерческой недвижимости (1311 чел.) или покупке жилой недвижимости для сдачи в наем (651 чел.).

Если обычная покупка жилой недвижимости относится к конечному потреблению, то приобретение коммерческой недвижимости или недвижимости для сдачи в наем относится к инвестиционным целям, т.е. этот кредит клиенты планируют не только вернуть, но вероятно еще и окупить. То есть клиенты планируют в ближайшем будущем получать положительный денежный поток. Доля таких клиентов составляет 9,1% от общего числа клиентов, поэтому ими не стоит пренебрегать, а стоит выделить отдельно.

### Категоризация данных

Ранее уже были созданы два вида категорий: один — по возрастам ('age_group'), другой — по целям ('purpose_category'). Но для ответа на поставленные вопросы потребуется провести еще одну категоризацию — по доходам. Для этого воспользуемся квантилями 25%, 50% и 75%.

In [37]:
statistics = df['total_income'].describe()
statistics[4:7]

25%    107620.0
50%    142594.0
75%    195818.0
Name: total_income, dtype: float64

- Функция для определения категории доходов.

In [38]:
def determine_income_group(income):
    if income <= statistics[4]: return 1
    elif statistics[4] < income <= statistics[5]: return 2
    elif statistics[5] < income <= statistics[6]: return 3
    else: return 4

- Применим функцию к столбцу доходов.

In [39]:
df['income_group'] = df['total_income'].apply(determine_income_group)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,34-43,недвижимость,4
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,34-43,автомобиль,2
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 34,недвижимость,3
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 34,образование,4
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,43-53,свадьба,3


**Вывод**

**Категоризация по возрастам**

Категоризация по возрастам ('age_group') была проведена при устранении пропущенных значений: в ней были использованы квантили (25%, 50% и 75%):

In [40]:
df['age_group'].value_counts()

age_group
до 34    5966
34-43    5276
53+      5275
43-53    4936
Name: count, dtype: int64

Больше всего клиентов в возрасте до 34 лет, их доля составляет 27,8%.

**Категоризация по целям**

Категоризация по целям ('purpose_category') была проведена ранее при лемматизации.

In [41]:
df['purpose_group'].value_counts()

purpose_group
недвижимость           8848
автомобиль             4306
образование            4013
свадьба                2324
инвестиционная цель    1962
Name: count, dtype: int64

Чаще всего клиенты берут кредит для приобретения недвижимости: доля таких клиентов из общего числа составляет 41,2%.

**Категоризация по доходам**

В качестве границ для категоризации доходов были использованы квантили (25%, 50% и 75%).

In [42]:
df['income_group'].value_counts()

income_group
2    5479
1    5364
4    5363
3    5247
Name: count, dtype: int64

Иначе говоря:

- доход 5364 человек меньше или равен 107,6 тыс. рублей
- доход 5479 человек находится в диапазоне от 107,6 до 142,6 тыс. рублей
- доход 5247 человек находится в диапазоне от 142,6 до 195,8 тыс. рублей
- 5364 человек зарабатывают больше 195,8 тыс. рублей

# 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
debt_from_children = df.groupby('children')['debt'].mean().reset_index()
debt_from_children.sort_values('debt', ascending = False)

,children,debt
4,4,0.097561
2,2,0.094925
1,1,0.091658
3,3,0.081818
0,0,0.075444
5,5,0.000000


**Вывод**

Чем больше детей в семье, тем больше просроченной задолженности. Но дети с тремя детьми чаще платят в срок, чем семьи с одним ребенком. А семьи без детей еще реже просрачивают платежи по кредиту, чем семьи с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
debt_from_family_status = df.groupby('family_status')['debt'].mean().reset_index()
debt_from_family_status.sort_values('debt', ascending = False)

,family_status,debt
0,Не женат / не замужем,0.097509
3,гражданский брак,0.093494
4,женат / замужем,0.075452
1,в разводе,0.071130
2,вдовец / вдова,0.065693


**Вывод**

Холостые и те кто живет в гражданском браке чаще просрачивают платежи, а те кто в разводе или овдовели чаще возвращают кредит в срок чем женатые/замужние.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
df_pivot = df.pivot_table(index = ['income_group'], columns = 'debt', values = 'gender', aggfunc = 'count')

- Посчитаем вероятность задолженности для каждой группы доходов

In [46]:
df_pivot['ratio'] = round(df_pivot[1] / (df_pivot[0] + df_pivot[1]), 3)
df_pivot.sort_values('ratio', ascending = False)

debt,0,1,ratio
income_group,,,
2,4996,483,0.088
3,4799,448,0.085
1,4937,427,0.080
4,4980,383,0.071


**Вывод**

Нет зависимости между уровнем дохода и возвратом кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [47]:
df_pivot = df.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'gender', aggfunc = 'count')

- Посчитаем вероятность задолженности для каждой группы доходов

In [48]:
df_pivot['ratio'] = round(df_pivot[1] / (df_pivot[0] + df_pivot[1]), 3)
df_pivot.sort_values('ratio', ascending = False)

debt,0,1,ratio
purpose_group,,,
автомобиль,3903,403,0.094
образование,3643,370,0.092
свадьба,2138,186,0.080
инвестиционная цель,1811,151,0.077
недвижимость,8217,631,0.071


**Вывод**

Получившиеся вероятности задолженности для каждой цели получения кредита, отсортированные по убыванию:

- автомобиль - 9,4%
- образование - 9,2%
- свадьба - 8,0%
- инвестиционная цель - 7,7%
- недвижимость - 7,1%

Самая низкая вероятность задолженности у клиентов, которые берут кредит для собственной недвижимости (покупка, строительство и т.д.). Люди настолько заинтересованы в собственном жилье, что платят даже ответственнее по сравнению с теми, кто берет кредит на инвестиционные цели (7,1% против 7,7%). Интересный факт: клиенты, берущие кредит на свадьбу, не самые безответственные плательщики из выборки – для них вероятность всего 8,0%. Хуже них платят только по образовательным и автокредитам.

# 4. Общий вывод

Профиль идеального клиента можно было бы изобразить следующим образом: это либо женатый мужчина, либо замужняя женщина (возможно, когда-то бывшие в браке), не имеющие детей, с месячным доходом свыше 196 тысяч рублей и целью кредитования — покупка собственного жилья. Такой портрет указывает на высокий уровень ответственности, что проявляется в статусе официального брака, достаточно высоком уровне дохода и отсутствии детей, что свидетельствует о наличии финансовых средств для обслуживания кредита, а также наличии конкретной материальной цели и отсутствии дополнительных финансовых обязательств.

В процессе анализа данных были обнаружены пропуски в столбцах, касающихся трудового стажа и дохода, каждый из которых составлял 2174 случая. Исследование показало, что пропуски в данных столбцах случайны и могут быть обусловлены тем, что люди на руководящих должностях или с образованием из престижных вузов чаще других не предоставляют информацию о своих доходах.

Учитывая случайный характер пропусков, возможные способы их корректировки включали удаление, взвешивание или заполнение пропусков. Хотя удаление данных (меньше одного процента от общего объема) было возможно, я выбрал заполнение пропусков медианными значениями для каждой подгруппы внутри столбца, поскольку информация — это ценный ресурс современности.